<a href="https://colab.research.google.com/github/zouidine/AMT_LLMs/blob/main/mixtral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zouidine/AMT_LLMs.git

Cloning into 'AMT_LLMs'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 25 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (25/25), 71.51 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.1 MB/s eta 0:00:00


In [3]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 731.0 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from nltk.translate.bleu_score import corpus_bleu
from langchain_groq import ChatGroq
from bert_score import BERTScorer
from tqdm import tqdm
import random

In [5]:
data_path = '/content/AMT_LLMs/data/'

ar_examples = open('{}ar_examples.txt'.format(data_path),
                   encoding='utf-8').read().strip().split('\n')

en_examples = open('{}en_examples.txt'.format(data_path),
                   encoding='utf-8').read().strip().split('\n')

ar_train_examples = open('{}kshots_ar_samples.txt'.format(data_path),
                         encoding='utf-8').read().strip().split('\n')

en_train_examples = open('{}kshots_en_samples.txt'.format(data_path),
                         encoding='utf-8').read().strip().split('\n')

#Zero-shot

###Arabic to English

####English Prompt

In [ ]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

system_message = "Translate the input text into English. Do not explain your answer!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(ar_examples))):
    prompt = [{"role":"system", "content":system_message},
     {"role":"user", "content":user_message_template.format(text=ar_examples[i])}]
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content)

100%|██████████| 500/500 [17:52<00:00,  2.15s/it]


In [ ]:
references = [[sen] for sen in en_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 43.3


In [ ]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='en')
_, _, F1 = scorer.score(tr_examples, en_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 70.24


####Arabic Prompt

In [6]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

system_message = "ترجم النص المدخل إلى اللغة الإنجليزية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(ar_examples))):
    prompt = [{"role":"system", "content":system_message},
     {"role":"user", "content":user_message_template.format(text=ar_examples[i])}]
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [17:20<00:00,  2.08s/it]


In [7]:
references = [[sen] for sen in en_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 48.21


In [9]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='en')
_, _, F1 = scorer.score(tr_examples, en_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 70.47


###English to Arabic

In [ ]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

system_message = "ترجم النص المدخل إلى اللغة العربية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(ar_examples))):
    prompt = [{"role":"system", "content":system_message},
     {"role":"user", "content":user_message_template.format(text=en_examples[i])}]
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [18:46<00:00,  2.25s/it]


In [ ]:
references = [[sen] for sen in ar_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 38.12


In [13]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='ar')
_, _, F1 = scorer.score(tr_examples, ar_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 77.2


#Few-shot

In [6]:
def create_examples(ar_samples, en_samples, ar2en=True, n=4):
    examples = []
    random.seed(42)
    ar = random.sample(ar_samples, n)
    random.seed(42)
    en = random.sample(en_samples, n)

    for i in range(n):
        if ar2en:
            examples.append({'text': ar[i], 'translation': en[i]})
        else:
            examples.append({'text': en[i], 'translation': ar[i]})
    return examples

def create_prompt(system_message, examples, user_message_template):
    few_shot_prompt = [{'role':'system', 'content':system_message}]
    for example in examples:
        example_input = example['text']
        example_translation = example['translation']
        few_shot_prompt.append(
            {'role':'user',
             'content':user_message_template.format(text=example_input)}
        )
        few_shot_prompt.append(
            {'role':'assistant', 'content':f"{example_translation}"}
        )
    return few_shot_prompt

##1-shot

####Arabic to English

In [29]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

examples = create_examples(ar_train_examples, en_train_examples, n=1)

system_message = "ترجم النص المدخل إلى اللغة الإنجليزية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(ar_examples))):
    prompt = create_prompt(system_message, examples, user_message_template)
    prompt.append({"role":"user",
                   "content":user_message_template.format(text=ar_examples[i])})
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [18:39<00:00,  2.24s/it]


In [30]:
references = [[sen] for sen in en_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 50.49


In [31]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='en')
_, _, F1 = scorer.score(tr_examples, en_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT Score:	 71.92


####English to Arabic

In [7]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

examples = create_examples(ar_train_examples, en_train_examples, ar2en=False,
                           n=1)

system_message = "ترجم النص المدخل إلى اللغة العربية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(en_examples))):
    prompt = create_prompt(system_message, examples, user_message_template)
    prompt.append({"role":"user",
                   "content":user_message_template.format(text=en_examples[i])})
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [18:52<00:00,  2.26s/it]


In [8]:
references = [[sen] for sen in ar_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 17.19


In [11]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='ar')
_, _, F1 = scorer.score(tr_examples, ar_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 77.37


##2-shot

####Arabic to English

In [36]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

examples = create_examples(ar_train_examples, en_train_examples, n=2)

system_message = "ترجم النص المدخل إلى اللغة الإنجليزية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(ar_examples))):
    prompt = create_prompt(system_message, examples, user_message_template)
    prompt.append({"role":"user",
                   "content":user_message_template.format(text=ar_examples[i])})
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [26:55<00:00,  3.23s/it]


In [37]:
references = [[sen] for sen in en_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 55.76


In [38]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='en')
_, _, F1 = scorer.score(tr_examples, en_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 73.71


####English to Arabic

In [40]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

examples = create_examples(ar_train_examples, en_train_examples, ar2en=False,
                           n=2)

system_message = "ترجم النص المدخل إلى اللغة العربية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(en_examples))):
    prompt = create_prompt(system_message, examples, user_message_template)
    prompt.append({"role":"user",
                   "content":user_message_template.format(text=en_examples[i])})
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [23:59<00:00,  2.88s/it]


In [41]:
references = [[sen] for sen in ar_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 39.53


In [42]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='ar')
_, _, F1 = scorer.score(tr_examples, ar_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 78.72


##3-shot

####Arabic to English

In [44]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

examples = create_examples(ar_train_examples, en_train_examples, n=3)

system_message = "ترجم النص المدخل إلى اللغة الإنجليزية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(ar_examples))):
    prompt = create_prompt(system_message, examples, user_message_template)
    prompt.append({"role":"user",
                   "content":user_message_template.format(text=ar_examples[i])})
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [39:01<00:00,  4.68s/it]


In [45]:
references = [[sen] for sen in en_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 56.48


In [46]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='en')
_, _, F1 = scorer.score(tr_examples, en_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 73.81


####English to Arabic

In [13]:
GROQ_API_KEY = "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"
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0,
                    api_key=GROQ_API_KEY)

examples = create_examples(ar_train_examples, en_train_examples, ar2en=False,
                           n=3)

system_message = "ترجم النص المدخل إلى اللغة العربية. لا تشرح إجابتك!"
user_message_template ="Text: {text}"

tr_examples = []
for i in tqdm(range(len(en_examples))):
    prompt = create_prompt(system_message, examples, user_message_template)
    prompt.append({"role":"user",
                   "content":user_message_template.format(text=en_examples[i])})
    response = groq_llm.invoke(prompt)
    tr_examples.append(response.content.split("\n")[0])

100%|██████████| 500/500 [36:48<00:00,  4.42s/it]


In [14]:
references = [[sen] for sen in ar_examples]
candidates = [sen for sen in tr_examples]
bleu = corpus_bleu(references, candidates)
print("BLEU Score:\t", round(100*bleu,2))

BLEU Score:	 41.66


In [15]:
scorer = BERTScorer(model_type='bert-base-uncased', lang='ar')
_, _, F1 = scorer.score(tr_examples, ar_examples)
print("BERT Score:\t", round(100*F1.mean().item(),2))

BERT Score:	 79.46
